In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer, AdamW
from datasets import load_dataset
from tqdm import tqdm
import torch

# Carica il modello preaddestrato e il tokenizzatore di BERT
model_name = r"C:\Users\drugo\Desktop\PROGETTO-IA-2\python\fine_tuned_bert_model"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')
dataset=load_dataset('squad',split='train')



In [ ]:
def predict_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt', max_length=512, truncation=True)
    start_scores, end_scores = model(**inputs).start_logits, model(**inputs).end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    all_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    answer = tokenizer.convert_tokens_to_string(all_tokens[start_index:end_index+1])

    return answer



In [ ]:
answer = dataset['answers']
answer=answer[:100]
print(len(answer))
lista_answer=[e['text']for e in answer]
lista_predicted=[]
for i in tqdm(range(0,100)):
    lista_predicted.append(predict_answer(dataset['question'][i], dataset['context'][i]))



In [ ]:
import evaluate
rouge=evaluate.load('rouge')
predictions=lista_predicted
references=lista_answer
results=rouge.compute(predictions=predictions,references=references)
print(results)